In [1]:
import pandas as pd
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pk

In [4]:
df=pd.read_csv(r'F:\Downloads\all_stocks_5yr.csv')

In [5]:
df1=df['close']
df1.iloc[0]

14.75

In [6]:
#Creating Environment Matrix 2x2x3
env_rows=2
env_cols=2
n_action=3

q_table=np.zeros((env_rows,env_cols,n_action))
np.random.seed()
pk.dump(q_table,open("pickl.pkl",'wb'))

In [7]:
pk.load(open("pickl.pkl",'rb'))

array([[[0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.]]])

In [8]:
#Defining Data Preprocessing Function

def data_prep(data,name):
    df=pd.DataFrame(data[data['Name']==name])
    df.dropna(inplace=True)
    df.drop(['high','low','volume','Name'],axis=1,inplace=True)
    df.reset_index(drop=True,inplace=True)
    # Calculating 5 day and 1 day Moving Average for DF
    df['5day_MA']=df['close'].rolling(5).mean()
    df['1day_MA']=df['close'].rolling(1).mean()
    df['5day_MA'][:4]=0
    #Splitting into train and Test data
    train_df=df[:int(len(df)*0.8)]
    test_df=df[int(len(df)*0.8):].reset_index(drop=True)
    return train_df,test_df

# Get the state for datapoint by Moving Average
def get_state(long_ma,short_ma,t):
    if short_ma<long_ma:
        if t==1:
            return (0,1) #Cash
        else :
            return (0,0) #Stock
    
    elif short_ma>long_ma:
        if t==1:
            return (1,1) #Cash
        else :
            return (1,0) #Stock


#Checking if the user can trade or not
def trade_t(num_of_stocks,port_value,current_price):
    if num_of_stocks>=0:
        if port_value>current_price:
            return 1
        else :return 0
    else:
        if port_value>current_price:
            return 1
        else :return 0



#Get next action by Epsilon greedy
def next_act(state,qtable,epsilon,action=3):
    if np.random.rand() < epsilon:
        action=np.random.randint(action)
    else:
        action=np.argmax(qtable[state])
        
        
    return action



# Immidiate reward Generator based on cummulative wealth 
def get_reward(state,action,current_close,past_close,buy_history):
    if state==(0,0) or state==(1,0): #Stock position
        if action==0:
            return -1000
        elif action==1:
            return (current_close-buy_history)
        elif action==2:
            return (current_close-past_close)
    
    elif state==(0,1) or state==(1,1): #Cash Position
        if action==0:
            return 0
        elif action==1:
            return -1000
        elif action==2:
            return (current_close-past_close)

## Reading and preprocessing the dataset

In [9]:
stocks=pd.read_csv(r'F:\Downloads\all_stocks_5yr.csv')
stocks_train,stocks_test=data_prep(stocks,'AAPL')

<ipython-input-8-2648047cb2ca>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['5day_MA'][:4]=0
C:\Users\DR. DEBASISH SAHA\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


## Training the dataset

In [10]:
episodes=100
port_value=1000
num_stocks=0
epsilon=1 #Epsilon Greedy
alpha=0.05 #Learning Rate
gamma=0.15 #Discount Factor
buy_history=0
net_worth=[1000] #Portfolio Value
np.random.seed()
for i in range(episodes): #Iteration for each episode
    port_value=1000
    num_stocks=0
    buy_history=0
    net_worth=[1000]
    

    for dt in range(len(stocks_train)): #Iteration through each dataset
        long_ma=stocks_train.iloc[dt]['5day_MA']
        short_ma=stocks_train.iloc[dt]['1day_MA']
        close_price=stocks_train.iloc[dt]['close']
        next_close=0
        
        if dt>0:
            past_close=stocks_train.iloc[dt-1]['close']
        else:
            past_close=close_price
        t=trade_t(num_stocks,net_worth[-1],close_price)
        state=get_state(long_ma,short_ma,t)
        action=next_act(state,q_table,epsilon)

        if action==0:#Buy
            
             num_stocks+=1
             buy_history=close_price
             to_append=net_worth[-1]-close_price
             net_worth.append(np.round(to_append,1))
             r=0
            
            
        
        elif action==1:#Sell
            # if num_stocks>0:
                num_stocks-=1               
                to_append=net_worth[-1]+close_price
                net_worth.append(np.round(to_append,1))
                # buy_history.pop(0)
        
        elif action==2:#hold
            to_append=net_worth[-1]+close_price
            net_worth.append(np.round(to_append,1))
            
        
                
         

        r=get_reward(state,action,close_price,past_close,buy_history) #Getting Reward
        
        try:
            next_state=get_state(stocks_train.iloc[dt+1]['5day_MA'],stocks_train.iloc[dt+1]['1day_MA'],t)
            
        except:
            break
        #Updating Q_table by Bellmen's Equation
        q_table[state][action]=(1.-alpha)*q_table[state][action]+alpha*(r+gamma*np.max(q_table[next_state]))
    
    if (epsilon-0.01)>0.15:
        epsilon-=0.01

print("Training Complete")

Training Complete


Training complete.

In [11]:
pk.dump(q_table,open('pickl.pkl','wb'))